In [1]:
#TODO Make the augmentation function for the dataset

In [2]:
#Importing necessary libraries and modules
import tensorflow as tf
import cv2 as cv
import numpy as np
import h5py

2024-06-25 16:55:52.449620: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 16:55:53.449407: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#Setting gpu growth rate to prevent OOM error
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

2024-06-25 16:55:57.911033: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-25 16:55:58.054323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-25 16:55:58.054834: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
#Data Extraction 
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 8
seed = 15
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/home/darklord/Downloads/hand_dataset/training_dataset/training_data/images/',
    labels = None,
    label_mode= None,
    batch_size= batch_size,
    seed=seed
)
raw_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/home/darklord/Downloads/hand_dataset/validation_dataset/validation_data/images/',
    labels = None,
    label_mode= None,
    batch_size= batch_size,
    seed=seed
)
raw_test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/home/darklord/Downloads/hand_dataset/test_dataset/test_data/images/',
    labels = None,
    label_mode= None,
    batch_size= batch_size,
    seed=seed
)

Found 4069 files belonging to 1 classes.
Found 738 files belonging to 1 classes.
Found 821 files belonging to 1 classes.


2024-06-25 17:03:05.856098: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-25 17:03:05.856681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-25 17:03:05.856864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
train_annotations_path = '/home/darklord/Downloads/hand_dataset/training_dataset/training_data/annotations/'
val_annotations_path = '/home/darklord/Downloads/hand_dataset/validation_dataset/validation_data/annotations/'
test_annotations_path = '/home/darklord/Downloads/hand_dataset/test_dataset/test_data/annotations/'


In [ ]:
#Data Augmentation , Caching and Prefetching for faster input pipeline
# data_augmentor = tf.keras.Sequential([
#     tf.keras.layers.RandomFlip("horizontal_and_vertical"),
#     tf.keras.layers.RandomRotation(0.2)
# ])
# train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = raw_val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# test_ds = raw_test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [1]:
%pip install scipy

  Using cached scipy-1.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 1.3 MB/s eta 0:00:00m eta 0:00:01 0:00:02
Note: you may need to restart the kernel to use updated packages.
